In [22]:
import requests
import numpy as np
import pandas as pd
from termcolor import colored
from ta.trend import ADXIndicator as ADX
import matplotlib.dates as mpdates
from datetime import date, timedelta
import plotly as py
import plotly.graph_objs as go
import ipywidgets
import time
import multiprocessing as mp
from multiprocessing import Process, Manager
import datetime
import pickle
import itertools
from itertools import permutations
import talib
import copy
import struct
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ta.momentum import RSIIndicator as Rsi
from ta.momentum import StochasticOscillator as SO
from ta.volatility import AverageTrueRange as ATRf
import ta
import json
from websocket import create_connection
import os
from talib.abstract import *
userid= "KQ7867"
pub_token="x7LmXOIZnLWnXdQ8ybUCLyi3ZkBk2IpK"
enc_token="mSlOF/mPRLmcVG9S6mWiYRYGF6jJ2J/iW9uCJ1bJx+0wC4MVGOSZpJHZtfU3r3ISTjWP6uiqIhZ0yCbtfWyynatFNZEnhuvn3edGkz6s0wA2TQVWOnN6RA=="
kf_session="FFIARlO7oXNBoPwot0s3beeVEzixmRmv"
n50={"RELIANCE":"738561","TCS":"2953217","HDFCBANK":"341249","INFY":"408065","HINDUNILVR":"356865",
     "ICICIBANK":"1270529","SBIN":"779521","BAJFINANCE":"81153","KOTAKBANK":"492033",
     "WIPRO":"969473","BHARTIARTL":"2714625","ASIANPAINT":"60417","HCLTECH":"1850625","ITC":"424961",
     "AXISBANK":"1510401","MARUTI":"2815745","LT":"2939649","ULTRACEMCO":"2952193","BAJAJFINSV":"4268801",
     "JSWSTEEL":"3001089","SUNPHARMA":"857857","TITAN":"897537","ONGC":"633601",
     "ADANIPORTS":"3861249","TATASTEEL":"895745","HDFCLIFE":"119553","POWERGRID":"3834113","TATAMOTORS":"884737",
     "BAJAJ-AUTO":"4267265","DIVISLAB":"2800641","NTPC":"2977281","TECHM":"3465729","ADANIENT":"6401",
     "BPCL":"134657","SBILIFE":"5582849","GRASIM":"315393","M&M":"519937",
     "DRREDDY":"225537","COALINDIA":"5215745","BRITANNIA":"140033","HINDALCO":"348929","CIPLA":"177665",
     "INDUSINDBK":"1346049","EICHERMOT":"232961","TATACONSUM":"878593","UPL":"2889473","HEROMOTOCO":"345089"}

nsid={"N50":"256265","NBANK":"260105","NIT":"259849","NAUTO":"263433","NPHARMA":"262409","NFMCG":"261897","NMETAL":"263689","NENEGRY":"261641"}    
#functions for getting datas
def get_fixed_minute_data(pid,sdate,edate):
    url="https://kite.zerodha.com/oms/instruments/historical/"+pid+"/minute" 
    x = requests.get(url,headers={"authorization":"enctoken "+enc_token}, params = {"oi":"1","from":sdate,"to":edate},cookies = {"kf_session":kf_session,"public_token":pub_token,
        "user_id" :userid,
        "enctoken": enc_token})
    xjson=x.json()   
    candles=xjson['data']['candles']
    
    return(candles)
def get_fixed_5min_data(pid,sdate,edate):
    url="https://kite.zerodha.com/oms/instruments/historical/"+pid+"/5minute" 
    x = requests.get(url,headers={"authorization":"enctoken "+enc_token}, params = {"user_id":"KQ7867","oi":"1","from":sdate,"to":edate},cookies = {"kf_session":kf_session,"public_token":pub_token,
        "user_id" :userid,
        "enctoken": enc_token})
    xjson=x.json()
    candles=xjson['data']['candles']
    return(candles)
def get_fixed_15min_data(pid,sdate,edate):
    url="https://kite.zerodha.com/oms/instruments/historical/"+pid+"/15minute" 
    x = requests.get(url,headers={"authorization":"enctoken "+enc_token}, params = {"oi":"1","from":sdate,"to":edate},cookies = {"kf_session":kf_session,"public_token":pub_token,
        "user_id" :userid,
        "enctoken": enc_token})
    xjson=x.json()
    candles=xjson['data']['candles']
    return(candles)
def get_fixed_30min_data(pid,sdate,edate):
    url="https://kite.zerodha.com/oms/instruments/historical/"+pid+"/30minute" 
    x = requests.get(url,headers={"authorization":"enctoken "+enc_token}, params = {"oi":"1","from":sdate,"to":edate},cookies = {"kf_session":kf_session,"public_token":pub_token,
        "user_id" :userid,
        "enctoken": enc_token})
    xjson=x.json()
    candles=xjson['data']['candles']
    return(candles)
def get_fixed_day_data(pid,sdate,edate):
    url="https://kite.zerodha.com/oms/instruments/historical/"+pid+"/day" 
    x = requests.get(url,headers={"authorization":"enctoken "+enc_token}, params = {"oi":"1","from":sdate,"to":edate},cookies = {"kf_session":kf_session,"public_token":pub_token,
        "user_id" :userid,
        "enctoken": enc_token})
    xjson=x.json()
    candles=xjson['data']['candles']
    return(candles)

#base for preprocessing
manager = Manager()
#time dictionary generation
now = datetime.datetime(2000, 1, 1, 9, 15, 0)
last = datetime.datetime(2000, 1, 1, 15, 30, 0)
delta = datetime.timedelta(minutes=1)

times = []
while now < last:
    times.append(now.strftime('%H:%M'))
    now += delta
time_canlen1_Dict={}
for ie ,i in enumerate(times):
    time_canlen1_Dict[i]=ie
#print(time_canlen1_Dict)
#function fetching fixed minute_data in real time parallel processing   
def mp_get_fixed_minute_data(company_list,sdate,edate,mul_script_data,time_canlen_Dict,check=False):
    now=datetime.datetime.now()    
    minute=now.minute
    hour =now.hour
    if (minute<10):
        minute="0"+str(minute)
    else:
        minute=str(minute)
    if (hour<10):
        hour="0"+str(hour)
    else:
        hour=str(hour)
    if (hour+":"+minute in time_canlen1_Dict.keys()):
        can_len=time_canlen1_Dict[hour+":"+minute]
        for c in company_list:
            company=c[0]
            pid=c[1]
            url="https://kite.zerodha.com/oms/instruments/historical/"+pid+"/minute" 
            x = requests.get(url,headers={"authorization":"enctoken "+enc_token}, 
                             params = {"oi":"1","from":sdate,"to":edate},
                             cookies = {"kf_session":kf_session,
                                        "public_token":pub_token,
                                        "user_id" :userid,
                                        "enctoken": enc_token})

            xjson=x.json()
            candles=xjson['data']['candles']
            if (check):
                print(x,xjson)
            while (len(candles)<can_len):
                print(len(candles),can_len)
                x = requests.get(url,headers={"authorization":"enctoken "+enc_token}, 
                             params = {"oi":"1","from":sdate,"to":edate},
                             cookies = {"kf_session":kf_session,
                                        "public_token":pub_token,
                                        "user_id" :userid,
                                        "enctoken": enc_token})

                xjson=x.json()
                candles=xjson['data']['candles']
            
            if (len(candles)==can_len):
                mul_script_data[company]=candles
            elif(len(candles)==can_len+1):
                mul_script_data[company]=candles[:-1]
            else:
                print("other else condition")
                pass
    else:
        print(str(now.hour)+":"+str(now.minute),"hi")
        pass
#used for getting older data and for test
def mp_get_fixed_all_minute_data_test(company_list,sdate,edate,mul_script_data,mul_script_5data,mul_script_30data,time_canlen_Dict):
        for c in company_list:
            company=c[0]
            pid=c[1]        
            url="https://kite.zerodha.com/oms/instruments/historical/"+pid+"/minute" 
            x = requests.get(url,headers={"authorization":"enctoken "+enc_token}, 
                             params = {"oi":"1","from":sdate,"to":edate},
                             cookies = {"kf_session":kf_session,
                                        "public_token":pub_token,
                                        "user_id" :userid,
                                        "enctoken": enc_token})

            xjson=x.json()
            candles=xjson['data']['candles']
            mul_script_data[company].extend(candles)
            
            url="https://kite.zerodha.com/oms/instruments/historical/"+pid+"/5minute" 
            x = requests.get(url,headers={"authorization":"enctoken "+enc_token}, 
                             params = {"oi":"1","from":sdate,"to":edate},
                             cookies = {"kf_session":kf_session,
                                        "public_token":pub_token,
                                        "user_id" :userid,
                                        "enctoken": enc_token})

            xjson=x.json()
            candles=xjson['data']['candles']
            mul_script_5data[company].extend(candles)
            
            url="https://kite.zerodha.com/oms/instruments/historical/"+pid+"/30minute" 
            x = requests.get(url,headers={"authorization":"enctoken "+enc_token}, 
                             params = {"oi":"1","from":sdate,"to":edate},
                             cookies = {"kf_session":kf_session,
                                        "public_token":pub_token,
                                        "user_id" :userid,
                                        "enctoken": enc_token})

            xjson=x.json()
            candles=xjson['data']['candles']
            mul_script_30data[company].extend(candles)
#fetch minute data parallel for test 
def mp_get_fixed_minute_data_test(company_list,sdate,edate,mul_script_data):
        
        for c in company_list:
            company=c[0]
            pid=c[1]        
            url="https://kite.zerodha.com/oms/instruments/historical/"+pid+"/minute" 
            x = requests.get(url,headers={"authorization":"enctoken "+enc_token}, 
                             params = {"oi":"1","from":sdate,"to":edate},
                             cookies = {"kf_session":kf_session,
                                        "public_token":pub_token,
                                        "user_id" :userid,
                                        "enctoken": enc_token})

            xjson=x.json()
            candles=xjson['data']['candles']
            mul_script_data[company]=candles
        print(company_list)
def is_support(df,i):  
    cond1 = df['Low'][i] < df['Low'][i-1]   
    cond2 = df['Low'][i] < df['Low'][i+1]   
    cond3 = df['Low'][i+1] < df['Low'][i+2]   
    cond4 = df['Low'][i-1] < df['Low'][i-2]  
    return (cond1 and cond2 and cond3 and cond4) 
# determine bearish fractal
def is_resistance(df,i):  
    cond1 = df['High'][i] > df['High'][i-1]   
    cond2 = df['High'][i] > df['High'][i+1]   
    cond3 = df['High'][i+1] > df['High'][i+2]   
    cond4 = df['High'][i-1] > df['High'][i-2]  
    return (cond1 and cond2 and cond3 and cond4)
# to make sure the new level area does not exist already
def is_far_from_level(value, levels, df):    
    ave =  np.mean(df['High'] - df['Low'])    
    return np.sum([abs(value-level)<ave for _,level in levels])==0
def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
    return next_month - datetime.timedelta(days=next_month.day)

def monthlist(begin,end):
    begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.month == 12:
            next_month = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_month = begin.replace(month=begin.month+1, day=1)
        if next_month > end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_month(begin).strftime("%Y-%m-%d")])
        begin = next_month
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result

def split_dates(begin,end,split_count):
    date_list = monthlist(begin,end)
    n = split_count
    all_dates=[]
    final = [date_list[i * n:(i + 1) * n] for i in range((len(date_list) + n - 1) // n )] 
    for f in final:
        all_dates.append([f[0][0],f[-1][-1]])
    return(all_dates)
def get_history_data(pid,sdate,edate,interval=None):
    if (interval==None):
        data=[]
        splitted_dates=split_dates(sdate,edate,3)
        for s in splitted_dates:
            data.extend(get_fixed_day_data(pid,s[0],s[1]))
        return(data)
    if (interval==5):
        data=[]
        splitted_dates=split_dates(sdate,edate,3)
        for s in splitted_dates:
            data.extend(get_fixed_5min_data(pid,s[0],s[1]))
        return(data)
    if (interval==1):
        data=[]
        splitted_dates=split_dates(sdate,edate,1)
        print(splitted_dates)
        for s in splitted_dates:
            print(s)
            data.extend(get_fixed_minute_data(pid,s[0],s[1]))
        return(data)
    if (interval==30):
        data=[]
        splitted_dates=split_dates(sdate,edate,5)
        for s in splitted_dates:
            data.extend(get_fixed_30min_data(pid,s[0],s[1]))
        return(data)

def mACO(close_prices):
    ma9=talib.MA(close_prices,timeperiod=9,matype=0)
    ma26=talib.MA(close_prices,timeperiod=26,matype=0)
    return([ma9,ma26])

def getFib(start,end):
    s=np.array([0,0.236,0.382,0.5,0.618,0.786,1])
    if (start>end):
        diff=start-end        
        return(start-(s*diff))
    else:
        diff=end-start
        return(start+(s*diff))
def getMinMax(candles):
    data=np.array(candles)
    high=max(data[:,2].astype("double"))
    ahigh=np.argmax(data[:,2].astype("double"))
    low=min(data[:,3].astype("double"))
    alow =np.argmin(data[:,3].astype("double"))
    return(low,high,alow,ahigh)
def isFib(candles):
    low,high,alow,ahigh=getMinMax(candles)
    levels=getFib(low,high)
    
    fcp=candles[-1][4]
    level=np.where(fcp>=levels)[0][-1]
    
    if (ahigh>alow):
        if (level==3 or level==4):
            return(True,abs(high-low)/low,level,"BUY")
    if (alow>ahigh):
        if (level==1 or level==2):
            return(True,abs(high-low)/low,level,"SELL")

def second_Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    return sub_li
def dayWiseSplitArgs(data):
    all_time=[s[s.find("T")+len("T"):s.rfind("+")] for s in np.array(data)[:,0]]
    args=np.where(np.array(all_time)=="09:15:00")
    return(args[0])
def gDWD(data):

    splitArgs=dayWiseSplitArgs(data)
    spData=[]
    for es in range(len(splitArgs)-1):
        spData.append(data[splitArgs[es]:splitArgs[es+1]])
    spData.append(data[splitArgs[-1]:])
    return(spData)
def datetostr(date):
    month=date.month
    day  =date.day
    if (month<10):
        month=str(month)
        month="0"+month
    else:
        month=str(month)
    if (day<10):
        day=str(day)
        day="0"+day
    else:
        day=str(day)
    date=str(date.year)+"-"+month+"-"+day
    return(date)
def getPreviousDates(date,reqDays,withHolidays):
    holidays={}
    holidaysD={}
    holidays["2022"]=["2022-01-26","2022-03-01","2022-03-18","2022-04-14","2022-04-15","2022-05-03","2022-08-09","2022-08-15","2022-08-31","2022-10-05","2022-10-26","2022-11-08",]
    holidays["2023"]=["2023-01-26","2023-03-07","2023-03-30", "2023-04-04", "2023-04-07", "2023-04-14", "2023-05-01", "2023-06-29",]
    holidaysD["2022"]=[]
    holidaysD["2023"]=[]
    for h in holidays["2022"]:
        holidaysD["2022"].append(datetime.datetime.strptime(h, "%Y-%m-%d"))
    for h in holidays["2023"]:
        holidaysD["2023"].append(datetime.datetime.strptime(h, "%Y-%m-%d"))
    
    dateD=datetime.datetime.strptime(date, "%Y-%m-%d")
    resultDates=[]
    if (withHolidays):
        while(reqDays>0):
            yesterday = dateD - timedelta(days = 1)
            if ((yesterday.weekday()<5)):
                resultDates.append(datetostr(yesterday))
                reqDays-=1
            dateD=yesterday
    else:
        while(reqDays>0):
            yesterday = dateD - timedelta(days = 1)
            if ((yesterday.weekday()<5) and (yesterday not in holidaysD["2022"]) and (yesterday not in holidaysD["2023"])):
                resultDates.append(datetostr(yesterday))
                reqDays-=1
            dateD=yesterday 
    
    return(resultDates)
def getDay(date):
    return(datetime.datetime.strptime(date, '%Y-%m-%d').weekday())
def getTwoWeekDates(date):
    monday=datetime.datetime.strptime(date, '%Y-%m-%d') - timedelta(days=getDay(date))
    dates = getPreviousDates(datetostr(monday),10,True)
    return(dates[-1],dates[-5],dates[4],dates[0])
def sortVwap(li,pos,des=False):
    li = np.array(li)
    diff = li[:,pos].astype("float")
    if (des):
        diff=diff*-1
    return (np.argsort(diff))
def vwapStd(df):
    q = df.volume.values
    highP= df.high.values
    lowP = df.low.values
    closeP=df.close.values
    df = df.assign(price=(highP+closeP+lowP)/3)
    p  = df.price.values
    df = df.assign(vwap=(p * q).cumsum() / q.cumsum())
    df = df.assign(v2sum = (p*p*q).cumsum())
    df = df.assign(volsum = q.cumsum())
    df = df.assign(stdDev = np.sqrt(df.v2sum.values/df.volsum.values-df.vwap.values*df.vwap.values))
    df = df.assign(vwap1SD = df.vwap.values+df.stdDev.values)
    df = df.assign(vwap2SD = df.vwap.values+2*df.stdDev.values)
    df = df.assign(vwap3SD = df.vwap.values+3*df.stdDev.values)
    df = df.assign(vwap1sd = df.vwap.values-df.stdDev.values)
    df = df.assign(vwap2sd = df.vwap.values-2*df.stdDev.values)
    df = df.assign(vwap3sd = df.vwap.values-3*df.stdDev.values)    
    return df
intSMap={0:"break",1:"sup4",2:"sup3",3:"sup2",4:"sup1",5:"bpp",6:"pp",7:"hpp",8:"res1",9:"res2",10:"res3",11:"res4"}
intRMap={0:"sup4",1:"sup3",2:"sup2",3:"sup1",4:"bpp",5:"pp",6:"hpp",7:"res1",8:"res2",9:"res3",10:"res4",11:"break"}

valSid={}

def unpack_int(bin, start, end, byte_format="I"):
    """Unpack binary data as unsgined interger."""
    return struct.unpack(">" + byte_format, bin[start:end])[0]
def split_packets( bin):
        """Split the data to individual packets of ticks."""
        # Ignore heartbeat data.
        if len(bin) < 2:
            return []

        number_of_packets = unpack_int(bin, 0, 2, byte_format="H")
        packets = []

        j = 2
        for i in range(number_of_packets):
            packet_length = unpack_int(bin, j, j + 2, byte_format="H")
            packets.append(bin[j + 2: j + 2 + packet_length])
            j = j + 2 + packet_length

        return packets
EXCHANGE_MAP = {
        "nse": 1,
        "nfo": 2,
        "cds": 3,
        "bse": 4,
        "bfo": 5,
        "bcd": 6,
        "mcx": 7,
        "mcxsx": 8,
        "indices": 9,
        # bsecds is replaced with it's official segment name bcd
        # so,bsecds key will be depreciated in next version
        "bsecds": 6,
    }
MODE_FULL = "full"
MODE_QUOTE = "quote"
MODE_LTP = "ltp"
def parse_binary( bin):
        """Parse binary data to a (list of) ticks structure."""
        packets = split_packets(bin)  # split data to individual ticks packet
        data = []

        for packet in packets:
            instrument_token = unpack_int(packet, 0, 4)
            segment = instrument_token & 0xff  # Retrive segment constant from instrument_token

            # Add price divisor based on segment
            if segment == EXCHANGE_MAP["cds"]:
                divisor = 10000000.0
            elif segment == EXCHANGE_MAP["bcd"]:
                divisor = 10000.0
            else:
                divisor = 100.0

            # All indices are not tradable
            tradable = False if segment == EXCHANGE_MAP["indices"] else True

            # LTP packets
            if len(packet) == 8:
                data.append({
                    "tradable": tradable,
                    "mode": MODE_LTP,
                    "instrument_token": instrument_token,
                    "last_price": unpack_int(packet, 4, 8) / divisor
                })
            # Indices quote and full mode
            elif len(packet) == 28 or len(packet) == 32:
                mode = MODE_QUOTE if len(packet) == 28 else MODE_FULL

                d = {
                    "tradable": tradable,
                    "mode": mode,
                    "instrument_token": instrument_token,
                    "last_price": unpack_int(packet, 4, 8) / divisor,
                    "ohlc": {
                        "high": unpack_int(packet, 8, 12) / divisor,
                        "low": unpack_int(packet, 12, 16) / divisor,
                        "open": unpack_int(packet, 16, 20) / divisor,
                        "close": unpack_int(packet, 20, 24) / divisor
                    }
                }

                # Compute the change price using close price and last price
                d["change"] = 0
                if(d["ohlc"]["close"] != 0):
                    d["change"] = (d["last_price"] - d["ohlc"]["close"]) * 100 / d["ohlc"]["close"]

                # Full mode with timestamp
                if len(packet) == 32:
                    try:
                        timestamp = datetime.fromtimestamp(unpack_int(packet, 28, 32))
                    except Exception:
                        timestamp = None

                    d["exchange_timestamp"] = timestamp

                data.append(d)
            # Quote and full mode
            elif len(packet) == 44 or len(packet) == 184:
                mode = MODE_QUOTE if len(packet) == 44 else MODE_FULL

                d = {
                    "tradable": tradable,
                    "mode": mode,
                    "instrument_token": instrument_token,
                    "last_price": unpack_int(packet, 4, 8) / divisor,
                    "last_traded_quantity": unpack_int(packet, 8, 12),
                    "average_traded_price": unpack_int(packet, 12, 16) / divisor,
                    "volume_traded": unpack_int(packet, 16, 20),
                    "total_buy_quantity": unpack_int(packet, 20, 24),
                    "total_sell_quantity": unpack_int(packet, 24, 28),
                    "ohlc": {
                        "open": unpack_int(packet, 28, 32) / divisor,
                        "high": unpack_int(packet, 32, 36) / divisor,
                        "low": unpack_int(packet, 36, 40) / divisor,
                        "close": unpack_int(packet, 40, 44) / divisor
                    }
                }

                # Compute the change price using close price and last price
                d["change"] = 0
                if(d["ohlc"]["close"] != 0):
                    d["change"] = (d["last_price"] - d["ohlc"]["close"]) * 100 / d["ohlc"]["close"]

                # Parse full mode
                if len(packet) == 184:
                    try:
                        last_trade_time = datetime.fromtimestamp(unpack_int(packet, 44, 48))
                    except Exception:
                        last_trade_time = None

                    try:
                        timestamp = datetime.fromtimestamp(unpack_int(packet, 60, 64))
                    except Exception:
                        timestamp = None

                    d["last_trade_time"] = last_trade_time
                    d["oi"] = unpack_int(packet, 48, 52)
                    d["oi_day_high"] = unpack_int(packet, 52, 56)
                    d["oi_day_low"] = unpack_int(packet, 56, 60)
                    d["exchange_timestamp"] = timestamp

                    # Market depth entries.
                    depth = {
                        "buy": [],
                        "sell": []
                    }

                    # Compile the market depth lists.
                    for i, p in enumerate(range(64, len(packet), 12)):
                        depth["sell" if i >= 5 else "buy"].append({
                            "quantity": unpack_int(packet, p, p + 4),
                            "price": unpack_int(packet, p + 4, p + 8) / divisor,
                            "orders": unpack_int(packet, p + 8, p + 10, byte_format="H")
                        })

                    d["depth"] = depth

                data.append(d)

        return data
page1={"ABBOTINDIA":"4583169","ADANIENT":"6401","APLAPOLLO":"6599681",
     "APLLTD":"6483969","ASHOKLEY":"54273","AUROPHARMA":"70401","BAJAJ-AUTO":"4267265",
     "BALKRISIND":"85761","BHARATFORG":"108033","BIOCON":"2911489","BOSCHLTD":"558337",
     "CIPLA":"177665","COALINDIA":"5215745","DIVISLAB":"2800641","DRREDDY":"225537",
     "EICHERMOT":"232961","GLAND":"303617","GLENMARK":"1895937","GRANULES":"3039233",
     "HEROMOTOCO":"345089","HINDALCO":"348929","HINDCOPPER":"4592385","HINDZINC":"364545",
     "IPCALAB":"418049","JINDALSTEL":"1723649","JSWSTEEL":"3001089","LAURUSLABS":"4923905",
     "LUPIN":"2672641","M&M":"519937","MARUTI":"2815745","MRF":"582913",
     "NATCOPHARM":"1003009","NATIONALUM":"1629185","NMDC":"3924993","PFIZER":"676609",
     "RATNAMANI":"3443457","SAIL":"758529","STAR":"1887745","SUNPHARMA":"857857",
     "TATAMOTORS":"884737","TATASTEEL":"895745","TIINDIA":"79873","TORNTPHARM":"900609",
     "TVSMOTOR":"2170625","VEDL":"784129","WELCORP":"3026177","ZYDUSLIFE":"2029825"}
page2={"AUBANK":"5436929","AXISBANK":"1510401","BAJAJFINSV":"4268801",
     "BAJFINANCE":"81153","BANDHANBNK":"579329","CHOLAFIN":"175361",
     "FEDERALBNK":"261889","HDFC":"340481","HDFCAMC":"1086465","HDFCBANK":"341249",
     "HDFCLIFE":"119553","ICICIBANK":"1270529","ICICIGI":"5573121",
     "ICICIPRULI":"4774913","IDFCFIRSTB":"2863105","INDUSINDBK":"1346049",
     "KOTAKBANK":"492033","M&MFIN":"3400961","MUTHOOTFIN":"6054401","PEL":"617473",
     "PFC":"3660545","PNB":"2730497","RBLBANK":"4708097","RECLTD":"3930881",
     "SBILIFE":"5582849","SBIN":"779521","SRTRANSFIN":"1102337"}
page3={"ADANIGREEN":"912129","ADANITRANS":"138049028","BPCL":"134657","BRITANNIA":"140033",
     "COFORGE":"2955009","COLPAL":"3876097","DABUR":"197633","EMAMILTD":"3460353",
     "GAIL":"1207553","GODREJCP":"2585345","HCLTECH":"1850625","HINDUNILVR":"356865",
     "INFY":"408065","IOC":"415745","ITC":"424961","LTI":"4561409","LTTS":"4752385",
     "MARICO":"1041153","MCDOWELL-N":"2674433","MINDTREE":"3675137","MPHASIS":"1152769",
     "NESTLEIND":"4598529","NTPC":"2977281","ONGC":"633601","PGHH":"648961",
     "POWERGRID":"3834113","RADICO":"2813441","RELIANCE":"738561","TATACONSUM":"878593",
     "TATAPOWER":"877057","TCS":"2953217","TECHM":"3465729","UBL":"4278529","VBL":"4843777",
     "WIPRO":"969473"}
n50={"RELIANCE":"738561","TCS":"2953217","HDFCBANK":"341249","INFY":"408065","HINDUNILVR":"356865",
     "ICICIBANK":"1270529","SBIN":"779521","BAJFINANCE":"81153","KOTAKBANK":"492033",
     "WIPRO":"969473","BHARTIARTL":"2714625","ASIANPAINT":"60417","HCLTECH":"1850625","ITC":"424961",
     "AXISBANK":"1510401","MARUTI":"2815745","LT":"2939649","ULTRACEMCO":"2952193","BAJAJFINSV":"4268801",
     "JSWSTEEL":"3001089","SUNPHARMA":"857857","TITAN":"897537","ONGC":"633601",
     "ADANIPORTS":"3861249","TATASTEEL":"895745","HDFCLIFE":"119553","POWERGRID":"3834113","TATAMOTORS":"884737",
     "BAJAJ-AUTO":"4267265","DIVISLAB":"2800641","NTPC":"2977281","TECHM":"3465729","ADANIENT":"6401",
     "BPCL":"134657","SBILIFE":"5582849","GRASIM":"315393","M&M":"519937",
     "DRREDDY":"225537","COALINDIA":"5215745","BRITANNIA":"140033","HINDALCO":"348929","CIPLA":"177665","APOLLOHOSP":"40193",
     "INDUSINDBK":"1346049","EICHERMOT":"232961","TATACONSUM":"878593","UPL":"2889473","HEROMOTOCO":"345089"}
sid ={ **n50}#, **page1,**page2,**page3} 
rsid={}
for s in sid.keys():
    rsid[sid[s]]=s
def zone_finder(pivots,value, classification=True, pivot=True, include_hbpp=True, only_current=True,absolute=True):
    if (pivot):
        piv= np.array([pivots['res5'],pivots['res4'],pivots['res3'],pivots['res2'],pivots['res1'],pivots['pp'],
          pivots['sup1'],pivots['sup2'],pivots['sup3'],pivots['sup4'],pivots['sup5']])
        piv_names= ["res5","res4","res3","res2","res1","pp","sup1","sup2","sup3","sup4","sup5"]
    else:
        piv = np.array([pivots["vwap3SD"],pivots["vwap2SD"],pivots["vwap1SD"], pivots["vwap"],
                       pivots["vwap1sd"],pivots["vwap2sd"],pivots["vwap3sd"]])
        piv_names= ["vwap3SD", "vwap2SD", "vwap1SD", "vwap", "vwap1sd", "vwap2sd","vwap3sd"]
    tmpupper=np.max(np.where(value<piv) if (len(np.where(value<piv)[0])>0) else [-1])#upper bound
    tmplower=np.min(np.where(value>piv) if (len(np.where(value>piv)[0])>0) else [-1])#lower bound
    upperBound=piv_names[tmpupper] if tmpupper!=-1 else None
    lowerBound=piv_names[tmplower] if tmplower!=-1 else None
    if (not(only_current)):
        if (upperBound in piv_names and upperBound!="res5"):
            next_upperBound = piv_names[tmpupper-1]
        else:
            next_upperBound = None
        if (lowerBound in piv_names and lowerBound!="sup5"):
            next_lowerBound = piv_names[tmplower +1]
    if classification:
        if (only_current):
            return(upperBound,lowerBound)
        else:
            return(next_upperBound,next_lowerBound)
    else:
        try:
            if (only_current):
                if (absolute):
                    return(round(pivots[upperBound],2),round(pivots[lowerBound],2))
                else:
                    return(round(((pivots[upperBound]-value)/value)*100,2),round(((value-pivots[lowerBound])/value)*100,2))
                
            else:
                return(round(((pivots[next_upperBound]-value)/value)*100,2),round(((value-pivots[next_lowerBound])/value)*100,2))
                
        except:
            return(None,None)

In [21]:
context=[['red_down_1_0', 'green_up_1_0'],
 ['red_up_1_0', 'red_up_1_1', 'green_down_1_0', 'red_down_2_0', 'green_down_2_0', 'green_down_2_1', 'red_down_3_0', 'red_down_3_1', 'red_down_3_2', 'red_up_3_3', 'green_down_3_0', 'green_down_3_1', 'green_up_3_2'],
 ['red_up_1_0', 'red_up_1_1', 'green_down_1_0', 'red_down_2_0', 'green_up_2_0'] ,
 ['red_up_1_0', 'green_down_1_0', 'red_up_2_0', 'red_up_2_1', 'red_down_2_2', 'green_down_2_0', 'green_up_2_1'], 
 ['green_down_0_1', 'red_down_1_0', 'green_down_1_0', 'green_up_1_1'], 
 ['green_down_0_1', 'red_up_1_0', 'green_down_1_0', 'green_up_1_1'], 
 ['green_down_0_1', 'green_down_0_2', 'green_down_0_3', 'red_up_1_0', 'green_down_1_0', 'green_down_1_1', 'red_up_2_0', 'green_down_2_0', 'red_down_3_0', 'green_down_3_0', 'green_down_3_1', 'red_up_4_0', 'green_down_4_0', 'red_up_5_0', 'red_up_5_1', 'green_down_5_0', 'red_up_6_0', 'green_up_6_0'] ,
 ['green_down_0_1', 'green_down_0_2', 'green_up_0_3'] ,
 ['green_down_0_1', 'green_up_0_2'] ,
 ['green_down_0_1', 'red_down_1_0', 'green_up_1_0'] ,
 ['green_down_0_1', 'green_down_0_2', 'green_up_0_3'] ,
 ['red_up_1_0', 'green_up_1_0'] ,
 ['red_up_1_0', 'red_down_1_1', 'red_down_1_2', 'red_up_1_3', 'green_down_1_0', 'red_up_2_0', 'red_up_2_1', 'red_down_2_2', 'green_down_2_0', 'red_down_3_0', 'green_down_3_0', 'green_down_3_1', 'green_up_3_2'], 
 ['green_down_0_1', 'green_down_0_2', 'red_up_1_0', 'green_down_1_0', 'green_down_1_1', 'green_down_1_2', 'green_down_1_3', 'red_down_2_0', 'red_up_2_1', 'green_down_2_0', 'green_down_2_1', 'red_down_3_0', 'green_down_3_0', 'red_up_4_0', 'green_up_4_0'], 
 ['red_down_1_0', 'red_up_1_1', 'red_up_1_2', 'green_down_1_0', 'green_up_1_1'] ,
 ['red_up_1_0', 'red_up_1_1', 'red_up_1_2', 'green_down_1_0', 'red_up_2_0', 'red_up_2_1', 'green_down_2_0', 'red_up_3_0', 'red_down_3_1', 'red_up_3_2', 'green_up_3_0'] ,
 ['red_up_1_0', 'green_up_1_0'] ,
 ['red_up_1_0', 'green_down_1_0', 'green_up_1_1'] ,
 ['green_down_0_1', 'green_up_0_2'] ,
 ['red_up_1_0', 'green_down_1_0', 'red_up_2_0', 'red_up_2_1', 'red_up_2_2', 'red_up_2_3', 'green_down_2_0', 'green_up_2_1'] ,
 ['red_up_1_0', 'red_up_1_1', 'red_up_1_2', 'red_down_1_3', 'red_down_1_4', 'red_up_1_5', 'green_up_1_0'] ,
 ['red_up_1_0', 'green_down_1_0', 'red_up_2_0', 'green_down_2_0', 'green_down_2_1', 'green_up_2_2'] ,
 ['green_down_0_1', 'red_up_1_0', 'green_down_1_0', 'green_down_1_1', 'red_up_2_0', 'red_up_2_1', 'green_down_2_0', 'green_down_2_1', 'red_up_3_0', 'green_down_3_0', 'red_up_4_0', 'red_up_4_1', 'red_down_4_2', 'green_down_4_0', 'green_down_4_1', 'red_up_5_0', 'red_up_5_1', 'red_up_5_2', 'red_up_5_3', 'red_up_5_4', 'red_down_5_5', 'green_down_5_0', 'red_down_6_0', 'green_up_6_0'] 
 ['red_up_1_0', 'green_down_1_0', 'red_up_2_0', 'red_up_2_1', 'green_down_2_0', 'green_down_2_1', 'red_up_3_0', 'red_down_3_1', 'red_up_3_2', 'red_down_3_3', 'red_up_3_4', 'green_down_3_0', 'green_down_3_1', 'red_up_4_0', 'red_up_4_1', 'green_down_4_0', 'red_up_5_0', 'green_down_5_0', 'green_up_5_1'] 
 ['green_down_0_1', 'red_up_1_0', 'red_up_1_1', 'green_up_1_0'] 
 ['red_up_1_0', 'green_down_1_0', 'red_down_2_0', 'green_down_2_0', 'green_down_2_1', 'red_up_3_0', 'red_up_3_1', 'red_down_3_2', 'red_up_3_3', 'red_up_3_4', 'red_up_3_5', 'green_down_3_0', 'green_down_3_1', 'red_up_4_0', 'red_up_4_1', 'green_down_4_0', 'red_down_5_0', 'green_up_5_0'] 
 ['red_up_1_0', 'green_down_1_0', 'red_up_2_0', 'red_up_2_1', 'red_down_2_2', 'red_up_2_3', 'red_up_2_4', 'green_down_2_0', 'red_up_3_0', 'red_down_3_1', 'green_up_3_0'] 
 ['red_up_1_0', 'red_up_1_1', 'red_down_1_2', 'green_up_1_0'] ,
 ['red_down_1_0', 'red_down_1_1', 'red_down_1_2', 'red_down_1_3', 'green_up_1_0'] ,
 ['red_up_1_0', 'red_up_1_1', 'green_up_1_0'] ,
 ['green_down_0_1', 'red_up_1_0', 'green_down_1_0', 'red_down_2_0', 'red_up_2_1', 'green_down_2_0', 'red_up_3_0', 'red_down_3_1', 'green_down_3_0', 'red_down_4_0', 'green_down_4_0', 'green_down_4_1', 'green_down_4_2', 'green_down_4_3', 'red_up_5_0', 'green_down_5_0', 'green_down_5_1', 'green_down_5_2', 'red_up_6_0', 'green_down_6_0', 'green_up_6_1'] 
 ['red_up_1_0', 'green_up_1_0'] ,
 ['red_down_1_0', 'green_up_1_0'] ,
 ['red_up_1_0', 'green_down_1_0', 'red_up_2_0', 'red_down_2_1', 'green_down_2_0', 'red_up_3_0', 'green_down_3_0', 'red_up_4_0', 'green_down_4_0', 'green_down_4_1', 'red_up_5_0', 'green_down_5_0', 'green_down_5_1', 'green_up_5_2'] ,
 ['green_down_0_1', 'red_down_1_0', 'red_down_1_1', 'green_down_1_0', 'red_up_2_0', 'green_down_2_0', 'green_up_2_1'] ,
 ['red_up_1_0', 'red_down_1_1', 'red_up_1_2', 'green_down_1_0', 'red_up_2_0', 'red_down_2_1', 'red_up_2_2', 'green_up_2_0'] ,
 ['red_up_1_0', 'red_down_1_1', 'green_down_1_0', 'green_up_1_1'] ,
 ['green_down_0_1', 'red_down_1_0', 'red_up_1_1', 'green_down_1_0', 'green_up_1_1'] ,
 ['red_up_1_0', 'green_down_1_0', 'red_up_2_0', 'green_down_2_0', 'red_up_3_0', 'green_up_3_0'] ,
 ['red_down_1_0', 'green_up_1_0'] ,
 ['red_up_1_0', 'red_up_1_1', 'green_down_1_0', 'green_down_1_1', 'red_up_2_0', 'green_down_2_0', 'green_up_2_1'] ,
 ['green_down_0_1', 'red_down_1_0', 'red_up_1_1', 'green_up_1_0'] ,
 ['red_up_1_0', 'green_down_1_0', 'green_up_1_1'] ]

<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
/tmp/ipykernel_3892/2235674887.py:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  context=[['red_down_1_0', 'green_up_1_0']
/tmp/ipykernel_3892/2235674887.py:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  context=[['red_down_1_0', 'green_up_1_0']
/tmp/ipykernel_3892/2235674887.py:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  context=[['red_down_1_0', 'green_up_1_0']
/tmp/ipykernel_3892/2235674887.py:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  context=[['red_down_1_0', 'green_up_1_0']
/tmp/ipykernel_3892/2235674887.py:1: SyntaxWarning: list indices must be integers or slices, n

TypeError: list indices must be integers or slices, not tuple

In [2]:
sdate = "2021-01-01"
tdate = "2024-06-01"
x=get_fixed_day_data(sid["WIPRO"],sdate,tdate )

In [ ]:
lol = get_fixed_15min_data(sid[s],xx[800][:10], xx[0][:10])

In [12]:
print(type(xx[0][:10]), xx[0][:10], x[0][0][:10])

<class 'str'> 2021-02-03 2021-01-01


In [34]:
intervals = ([0,100],[100, 200],
            [200,300],[300, 400],
            [400,500],[500, 600],
            [600,700],[700, 800],
            [800, -1])

In [38]:
rsid={}
completed = []    
for s in sid.keys():
        print(s)
        full_data = []
        for interval in intervals:
            day_data = get_fixed_15min_data(sid[s],x[interval[0]][0][:10], x[interval[-1]][0][:10])
            full_data.extend(day_data)
        with open("all_data/"+s, "wb") as fp:   #Pickling
            pickle.dump(full_data, fp)

RELIANCE
TCS
HDFCBANK
INFY
HINDUNILVR
ICICIBANK
SBIN
BAJFINANCE
KOTAKBANK
WIPRO
BHARTIARTL
ASIANPAINT
HCLTECH
ITC
AXISBANK
MARUTI
LT
ULTRACEMCO
BAJAJFINSV
JSWSTEEL
SUNPHARMA
TITAN
ONGC
ADANIPORTS
TATASTEEL
HDFCLIFE
POWERGRID
TATAMOTORS
BAJAJ-AUTO
DIVISLAB
NTPC
TECHM
ADANIENT
BPCL
SBILIFE
GRASIM
M&M
DRREDDY
COALINDIA
BRITANNIA
HINDALCO
CIPLA
APOLLOHOSP
INDUSINDBK
EICHERMOT
TATACONSUM
UPL
HEROMOTOCO


In [3]:
s="RELIANCE"
with open("all_data/"+s, "rb") as fp:   # Unpickling
    b = pickle.load(fp)
    df = pd.DataFrame(b, columns=['date', 'open', 'high', 'low', 'close', 'volume', 'un'])
        

In [25]:
df_all={}
for s in sid.keys():
    
    with open("all_data/"+s, "rb") as fp:   # Unpickling
        b = pickle.load(fp)
        df_all[s] = pd.DataFrame(b, columns=['date', 'open', 'high', 'low', 'close', 'volume', 'un'])
            

In [26]:
for s in sid.keys():
    df_all[s].drop_duplicates(inplace=True, ignore_index=True)
    df_all[s]["str_date"]=df_all[s]["date"]
    df_all[s]["date"]=pd.to_datetime(df_all[s]["str_date"])
    df_all[s]["rsi"]=ta.momentum.rsi(df_all[s].loc[:,'close'] , window=14)

In [214]:
for ei, i in enumerate(df["str_date"]):
    if ("2022-03-21" in i):
        print(df.iloc[ei])

open                          2329.6
high                          2362.3
low                           2329.6
close                        2355.65
volume                        973749
un                                 0
str_date    2022-03-21T09:15:00+0530
rsi                        69.182553
Name: 2022-03-21 09:15:00+05:30, dtype: object
open                         2355.65
high                          2357.5
low                           2346.8
close                        2351.55
volume                        322527
un                                 0
str_date    2022-03-21T09:30:00+0530
rsi                        65.371544
Name: 2022-03-21 09:30:00+05:30, dtype: object
open                          2351.1
high                          2351.1
low                           2340.1
close                        2340.45
volume                        273066
un                                 0
str_date    2022-03-21T09:45:00+0530
rsi                        56.325257
Name: 2022-03-21 0

In [20]:
#get the first date
for each_case in range(len(all_sdate)):
    start_arrow = df_all[s].loc[all_sdate[each_case]]["close"]
    end_arrow = df_all[s].loc[all_edate[each_case]]["close"]
    peak_of_arrow = max(df_all[s][all_sdate[each_case]:all_edate[each_case]]["close"])
    rev_context = all_rev_context[each_case]
    all_resdates = []
    all_resdates.extend(all_rsdate[each_case])
    all_resdates.extend(all_redate[each_case])
    all_resdates = sorted(all_resdates)
    all_resdates = all_resdates[::-1]
    sgot_context = False
    egot_context = False
    pgot_context = False
    all_context = []
    all_context_val = []
    for ea, a in enumerate(all_resdates[:-1]):
        # print(all_resdates[ea], all_resdates[ea+1])
        strip =df_all[s]["close"][(df_all[s].index<=all_resdates[ea]) & (df_all[s].index>=all_resdates[ea+1])]
        for ind, val in strip.iloc[::-1].items():
            if (val>start_arrow):
                # print(ea, "ind=",ind)
                if (ea%2):
                    ea-=1
                context_index = ea//2
                all_context.append(rev_context[context_index])
                all_context_val.append(round(df_all[s].loc[ind]["rsi"], 2))
                sgot_context = True
                break
        if sgot_context:
            break   
    if (not(sgot_context)):
        all_context.append("NA")
        all_context_val.append("NA")
    for ea, a in enumerate(all_resdates[:-1]):
        # print(all_resdates[ea], all_resdates[ea+1])
        strip =df_all[s]["close"][(df_all[s].index<=all_resdates[ea]) & (df_all[s].index>=all_resdates[ea+1])]
        for ind, val in strip.iloc[::-1].items():
            if (val>end_arrow):
                if (ea%2):
                    ea-=1
                context_index = ea//2
                all_context.append(rev_context[context_index])
                all_context_val.append(round(df_all[s].loc[ind]["rsi"], 2))
                egot_context = True
                break
        if egot_context:
            break    
    if (not(egot_context)):
        # all_context.append(("NA", "NA"))
        all_context.append("NA")
        all_context_val.append("NA")

    
    for ea, a in enumerate(all_resdates[:-1]):
        # print(all_resdates[ea], all_resdates[ea+1])
        strip =df_all[s]["close"][(df_all[s].index<=all_resdates[ea]) & (df_all[s].index>=all_resdates[ea+1])]
        for ind, val in strip.iloc[::-1].items():
            if (val>peak_of_arrow):
                if (ea%2):
                    ea-=1
                context_index = ea//2
                all_context.append(rev_context[context_index])
                all_context_val.append(round(df_all[s].loc[ind]["rsi"], 2))
                pgot_context = True
                break
        if pgot_context:
            break    
    if (not(pgot_context)):
        # all_context.append(("NA", "NA"))
        all_context.append("NA")
        all_context_val.append("NA")
    print(all_sdate[each_case], "&", all_edate[each_case], "&", all_rsdate[each_case],"&",  all_redate[each_case],"&", rev_context,"&",all_context[0], "&", all_context[1],"&", all_context[2], "&",all_context_val[0], "&", all_context_val[1],"&", all_context_val[2] )


2021-01-12 13:00:00+05:30 & 2021-01-13 11:45:00+05:30 & [Timestamp('2021-01-11 13:45:00+0530', tz='UTC+05:30'), Timestamp('2021-01-08 15:00:00+0530', tz='UTC+05:30'), Timestamp('2021-01-06 09:15:00+0530', tz='UTC+05:30'), Timestamp('2021-01-04 13:45:00+0530', tz='UTC+05:30'), Timestamp('2021-01-04 09:45:00+0530', tz='UTC+05:30')] & [Timestamp('2021-01-12 09:15:00+0530', tz='UTC+05:30'), Timestamp('2021-01-11 10:15:00+0530', tz='UTC+05:30'), Timestamp('2021-01-07 09:15:00+0530', tz='UTC+05:30'), Timestamp('2021-01-05 09:15:00+0530', tz='UTC+05:30'), Timestamp('2021-01-04 13:30:00+0530', tz='UTC+05:30')] & ['red_up_1_0', 'green_down_1_0', 'red_down_2_0', 'green_down_2_0', 'red_up_3_0'] & green_down_1_0 & red_down_2_0 & red_down_2_0 & 58.72 & 29.88 & 40.12
2021-01-20 13:00:00+05:30 & 2021-01-22 09:45:00+05:30 & [Timestamp('2021-01-19 09:15:00+0530', tz='UTC+05:30'), Timestamp('2021-01-15 14:00:00+0530', tz='UTC+05:30'), Timestamp('2021-01-12 13:00:00+0530', tz='UTC+05:30')] & [Timestamp('

In [138]:
df["date"]<all_sdate[0] 

0         True
1         True
2         True
3         True
4         True
         ...  
21233    False
21234    False
21235    False
21236    False
21237    False
Name: date, Length: 21238, dtype: bool

In [27]:
for s in sid.keys():
    Upeak = False
    Lpeak = False
    crossed_60 = True
    crossed_40 = True
    
    gcontext_arrow_directions = []
    gcontext_arrow_closePrices = []
    gcontext_arrow_rsi = []
    
    all_sdate=[]
    all_edate=[]
    all_rev_context=[]
    all_rsdate=[]
    all_redate=[]

    for ei, i in enumerate(df_all[s]["rsi"]):
        if(not (Upeak)) and (i>69):
            Upeak = True
            crossed_60 = False
            start_price = df_all[s]["close"][ei]
            start_date = df_all[s]["date"][ei]
            dotted_price = df_all[s]["close"][ei]
        if (Upeak) and (not(crossed_60)):
            if (i<60):
                crossed_60 = True
        if (Upeak and crossed_60):
            if (i>69):
                start_price = df_all[s]["close"][ei]
                start_date = df_all[s]["date"][ei]
                crossed_60 = False
        if (Upeak and i<50):
            Upeak = False
            end_price = df_all[s]["close"][ei]
            end_date = df_all[s]["date"][ei]
                
            if (end_price >= start_price):
                gen = 0
                prev="green"
                rev_context_list = []
                rev_close_price =[]
                rev_rsi = []
                rev_sdates = []
                rev_edates = []
                sub_gen = 0
                for c in gcontext_arrow_directions[::-1]:
                    if "red" in c[0] :
                        if prev=="green":
                            gen+=1
                            prev="red"
                            sub_gen=0
                        else:
                            sub_gen +=1
                    if "green" in c[0]:
                        if prev=="red":
                            prev = "green"
                            sub_gen = 0
                        else:
                            sub_gen +=1
                    rev_context_list.append(c[0]+"_"+str(gen)+"_"+str(sub_gen))
                    rev_sdates.append(c[1])
                    rev_edates.append(c[2])
                for c in gcontext_arrow_closePrices[::-1]:
                    rev_close_price.append(c)
                for c in gcontext_arrow_rsi[::-1]:
                    rev_rsi.append(c)
                # print(s, "&", start_date, "&", df_all[s]["date"][ei], "&",rev_context_list, "&",rev_sdates, "&",rev_edates)
                all_sdate.append(start_date)
                all_edate.append(df_all[s]["date"][ei])
                all_rev_context.append(rev_context_list)
                all_rsdate.append(rev_sdates)
                all_redate.append(rev_edates)
    
                gcontext_arrow_directions= [["green_up",start_date, df_all[s]["date"][ei]]]
                gcontext_arrow_closePrices.append(df_all[s]["close"][ei])
                gcontext_arrow_rsi.append(df_all[s]["rsi"][ei])
            else:
                gcontext_arrow_directions.append(["green_down", start_date, df_all[s]["date"][ei]])
                gcontext_arrow_closePrices.append(df_all[s]["close"][ei])
                gcontext_arrow_rsi.append(df_all[s]["rsi"][ei])
    
        if (not(Lpeak)) and (i<31):
            Lpeak = True
            crossed_40 = False
            start_price = df_all[s]["close"][ei]
            start_date = df_all[s]["date"][ei]
            
        if (Lpeak) and (not(crossed_40)):
            if (i>40):
                crossed_40 = True
        if (Lpeak and crossed_40):
            if (i<31):
                start_price = df_all[s]["close"][ei]
                start_date = df_all[s]["date"][ei]
                crossed_40 = False
        if (Lpeak and i>50):
            Lpeak = False
            end_price = df_all[s]["close"][ei]
            end_date = df_all[s]["date"][ei]
                
            if (end_price <= start_price):
                # print(start_date, df["date"][ei], gcontext_arrow_directions)
                gcontext_arrow_directions.append(["red_down",start_date, df_all[s]["date"][ei]])
                gcontext_arrow_closePrices.append(df_all[s]["close"][ei])
                gcontext_arrow_rsi.append(df_all[s]["rsi"][ei])
            else:
                gcontext_arrow_directions.append(["red_up",start_date, df_all[s]["date"][ei]])
                gcontext_arrow_closePrices.append(df_all[s]["close"][ei])
                gcontext_arrow_rsi.append(df_all[s]["rsi"][ei])
    df_all[s]=df_all[s].set_index("date")
    #get the first date
    for each_case in range(len(all_sdate)):
        start_arrow = df_all[s].loc[all_sdate[each_case]]["close"]
        end_arrow = df_all[s].loc[all_edate[each_case]]["close"]
        peak_of_arrow = max(df_all[s][all_sdate[each_case]:all_edate[each_case]]["close"])
        rev_context = all_rev_context[each_case]
        all_resdates = []
        all_resdates.extend(all_rsdate[each_case])
        all_resdates.extend(all_redate[each_case])
        all_resdates = sorted(all_resdates)
        all_resdates = all_resdates[::-1]
        sgot_context = False
        egot_context = False
        pgot_context = False
        all_context = []
        all_context_val = []
        for ea, a in enumerate(all_resdates[:-1]):
            # print(all_resdates[ea], all_resdates[ea+1])
            strip =df_all[s]["close"][(df_all[s].index<=all_resdates[ea]) & (df_all[s].index>=all_resdates[ea+1])]
            for ind, val in strip.iloc[::-1].items():
                if (val>start_arrow):
                    # print(ea, "ind=",ind)
                    if (ea%2):
                        ea-=1
                    context_index = ea//2
                    all_context.append(rev_context[context_index])
                    all_context_val.append(round(df_all[s].loc[ind]["rsi"], 2))
                    sgot_context = True
                    break
            if sgot_context:
                break   
        if (not(sgot_context)):
            all_context.append("NA")
            all_context_val.append("NA")
        for ea, a in enumerate(all_resdates[:-1]):
            # print(all_resdates[ea], all_resdates[ea+1])
            strip =df_all[s]["close"][(df_all[s].index<=all_resdates[ea]) & (df_all[s].index>=all_resdates[ea+1])]
            for ind, val in strip.iloc[::-1].items():
                if (val>end_arrow):
                    if (ea%2):
                        ea-=1
                    context_index = ea//2
                    all_context.append(rev_context[context_index])
                    all_context_val.append(round(df_all[s].loc[ind]["rsi"], 2))
                    egot_context = True
                    break
            if egot_context:
                break    
        if (not(egot_context)):
            # all_context.append(("NA", "NA"))
            all_context.append("NA")
            all_context_val.append("NA")
    
        
        for ea, a in enumerate(all_resdates[:-1]):
            # print(all_resdates[ea], all_resdates[ea+1])
            strip =df_all[s]["close"][(df_all[s].index<=all_resdates[ea]) & (df_all[s].index>=all_resdates[ea+1])]
            for ind, val in strip.iloc[::-1].items():
                if (val>peak_of_arrow):
                    if (ea%2):
                        ea-=1
                    context_index = ea//2
                    all_context.append(rev_context[context_index])
                    all_context_val.append(round(df_all[s].loc[ind]["rsi"], 2))
                    pgot_context = True
                    break
            if pgot_context:
                break    
        if (not(pgot_context)):
            # all_context.append(("NA", "NA"))
            all_context.append("NA")
            all_context_val.append("NA")
        print(s, "&", all_sdate[each_case], "&", all_edate[each_case], "&", all_rsdate[each_case],"&",  all_redate[each_case],"&", rev_context,"&",all_context[0], "&", all_context[1],"&", all_context[2], "&",all_context_val[0], "&", all_context_val[1],"&", all_context_val[2] )


RELIANCE & 2021-01-12 13:00:00+05:30 & 2021-01-13 11:45:00+05:30 & [Timestamp('2021-01-11 13:45:00+0530', tz='UTC+05:30'), Timestamp('2021-01-08 15:00:00+0530', tz='UTC+05:30'), Timestamp('2021-01-06 09:15:00+0530', tz='UTC+05:30'), Timestamp('2021-01-04 13:45:00+0530', tz='UTC+05:30'), Timestamp('2021-01-04 09:45:00+0530', tz='UTC+05:30')] & [Timestamp('2021-01-12 09:15:00+0530', tz='UTC+05:30'), Timestamp('2021-01-11 10:15:00+0530', tz='UTC+05:30'), Timestamp('2021-01-07 09:15:00+0530', tz='UTC+05:30'), Timestamp('2021-01-05 09:15:00+0530', tz='UTC+05:30'), Timestamp('2021-01-04 13:30:00+0530', tz='UTC+05:30')] & ['red_up_1_0', 'green_down_1_0', 'red_down_2_0', 'green_down_2_0', 'red_up_3_0'] & green_down_1_0 & red_down_2_0 & red_down_2_0 & 58.72 & 29.88 & 40.12
RELIANCE & 2021-01-20 13:00:00+05:30 & 2021-01-22 09:45:00+05:30 & [Timestamp('2021-01-19 09:15:00+0530', tz='UTC+05:30'), Timestamp('2021-01-15 14:00:00+0530', tz='UTC+05:30'), Timestamp('2021-01-12 13:00:00+0530', tz='UTC+0

In [29]:
rev_context

['red_up_1_0',
 'green_down_1_0',
 'red_up_2_0',
 'green_down_2_0',
 'green_down_2_1',
 'green_down_2_2',
 'red_down_3_0',
 'red_down_3_1',
 'red_down_3_2',
 'red_down_3_3',
 'green_down_3_0',
 'red_up_4_0',
 'red_down_4_1',
 'green_down_4_0',
 'red_up_5_0',
 'green_up_5_0']

In [151]:
possible_context = ["red_up", "red_down", "green_down", "green_up"]

for r in rev_context:
    
    

,open,high,low,close,volume,un,str_date,rsi
date,,,,,,,,
2021-01-01 09:15:00+05:30,1894.95,1899.95,1893.05,1895.65,593944,0,2021-01-01T09:15:00+0530,NaN
2021-01-01 09:30:00+05:30,1895.65,1896.60,1889.45,1893.55,341123,0,2021-01-01T09:30:00+0530,NaN
2021-01-01 09:45:00+05:30,1893.65,1894.95,1892.10,1893.80,150091,0,2021-01-01T09:45:00+0530,NaN
2021-01-01 10:00:00+05:30,1894.00,1894.95,1892.60,1893.70,114342,0,2021-01-01T10:00:00+0530,NaN
2021-01-01 10:15:00+05:30,1893.75,1898.30,1893.25,1895.65,246469,0,2021-01-01T10:15:00+0530,NaN
...,...,...,...,...,...,...,...,...
2024-05-31 14:15:00+05:30,2859.95,2861.40,2849.60,2853.90,327735,0,2024-05-31T14:15:00+0530,45.860384
2024-05-31 14:30:00+05:30,2854.30,2857.80,2845.00,2854.80,335289,0,2024-05-31T14:30:00+0530,46.913181
2024-05-31 14:45:00+05:30,2854.80,2855.65,2846.20,2854.50,379639,0,2024-05-31T14:45:00+0530,46.587969


In [102]:
type(all_sdate[-1])

pandas._libs.tslibs.timestamps.Timestamp

In [108]:
df[0]

KeyError: 0

In [97]:
df[all_sdate[-1]]

KeyError: Timestamp('2024-05-23 13:15:00+0530', tz='UTC+05:30')

In [89]:
df[(df["date"]>=all_sdate[-1]) & (df["date"]<=all_edate[-1])]["close"][0]

KeyError: 0

In [83]:
print(df["date"]<all_sdate[0]) 
print(df["date"]>all_edate[0])

0         True
1         True
2         True
3         True
4         True
         ...  
21233    False
21234    False
21235    False
21236    False
21237    False
Name: date, Length: 21238, dtype: bool
0        False
1        False
2        False
3        False
4        False
         ...  
21233     True
21234     True
21235     True
21236     True
21237     True
Name: date, Length: 21238, dtype: bool


In [78]:
df[df["date"]>all_sdate[0] & df["date"]<all_edate[0]]

TypeError: unsupported operand type(s) for &: 'Timestamp' and 'DatetimeArray'

In [66]:
df.set_index("date")

,date,open,high,low,close,volume,un,rsi
0,2021-01-01T09:15:00+0530,1894.95,1899.95,1893.05,1895.65,593944,0,NaN
1,2021-01-01T09:30:00+0530,1895.65,1896.60,1889.45,1893.55,341123,0,NaN
2,2021-01-01T09:45:00+0530,1893.65,1894.95,1892.10,1893.80,150091,0,NaN
3,2021-01-01T10:00:00+0530,1894.00,1894.95,1892.60,1893.70,114342,0,NaN
4,2021-01-01T10:15:00+0530,1893.75,1898.30,1893.25,1895.65,246469,0,NaN
...,...,...,...,...,...,...,...,...
21233,2024-05-31T14:15:00+0530,2859.95,2861.40,2849.60,2853.90,327735,0,45.860384
21234,2024-05-31T14:30:00+0530,2854.30,2857.80,2845.00,2854.80,335289,0,46.913181
21235,2024-05-31T14:45:00+0530,2854.80,2855.65,2846.20,2854.50,379639,0,46.587969
21236,2024-05-31T15:00:00+0530,2854.45,2871.35,2848.30,2868.40,3840856,0,60.314994


In [178]:
df.index

DatetimeIndex(['2021-01-01 09:15:00+05:30', '2021-01-01 09:30:00+05:30',
               '2021-01-01 09:45:00+05:30', '2021-01-01 10:00:00+05:30',
               '2021-01-01 10:15:00+05:30', '2021-01-01 10:30:00+05:30',
               '2021-01-01 10:45:00+05:30', '2021-01-01 11:00:00+05:30',
               '2021-01-01 11:15:00+05:30', '2021-01-01 11:30:00+05:30',
               ...
               '2024-05-31 13:00:00+05:30', '2024-05-31 13:15:00+05:30',
               '2024-05-31 13:30:00+05:30', '2024-05-31 13:45:00+05:30',
               '2024-05-31 14:00:00+05:30', '2024-05-31 14:15:00+05:30',
               '2024-05-31 14:30:00+05:30', '2024-05-31 14:45:00+05:30',
               '2024-05-31 15:00:00+05:30', '2024-05-31 15:15:00+05:30'],
              dtype='datetime64[ns, UTC+05:30]', name='date', length=21238, freq=None)

In [23]:
a =[1,2,3,4,5,6,7,8,9,10]
for ei, i in enumerate(a):
    print("pivot=", i)
    for ej, j in enumerate(a[ei+1:]):
        print(a[ei+ej+1])

pivot= 1
2
3
4
5
6
7
8
9
10
pivot= 2
3
4
5
6
7
8
9
10
pivot= 3
4
5
6
7
8
9
10
pivot= 4
5
6
7
8
9
10
pivot= 5
6
7
8
9
10
pivot= 6
7
8
9
10
pivot= 7
8
9
10
pivot= 8
9
10
pivot= 9
10
pivot= 10


In [43]:
context = ['red_up', 'red_up', 'red_up', 'green_down', 'red_up', 'red_up', 'red_up', 'green_down', 'red_down', 'green_down', 'red_up', 'green_down']
# context = ['green_down', 'red_up']
context = ['green_down', 'green_down', 'red_up', 'red_down', 'red_down', 'red_down', 'green_down', 'green_down', 'red_down', 'green_down', 'red_up', 'red_up']

In [44]:
gen = 0
prev="green"
rev_context_list = []
for c in context[::-1]:
    if "red" in c :
        if prev=="green":
            gen+=1
            prev="red"
            sub_gen=0
        else:
            sub_gen +=1
    if "green" in c:
        if prev=="red":
            prev = "green"
            sub_gen = 0
        else:
            sub_gen +=1
    rev_context_list.append(c+"_"+str(gen)+"_"+str(sub_gen))

red_up_1_0
red_up_1_1
green_down_1_0
red_down_2_0
green_down_2_0
green_down_2_1
red_down_3_0
red_down_3_1
red_down_3_2
red_up_3_3
green_down_3_0
green_down_3_1


In [31]:
stack = []
count = 0
for ei, i in enumerate(df["rsi"]):
    if (ei>1984):
#         print(df["date"][ei])
        if (df["rsi"][ei-1]<70 and i >70):
            if (len(stack)==0):
                stack.append(df.iloc[ei]["date"][:11])
            else:
                if (stack[-1]!=df.iloc[ei]["date"][:11]):
                    stack.append(df.iloc[ei]["date"][:11])
        if (i<=30):
            if (len(stack) >=3):
                print(count, stack)
                count += 1
            stack = []

0 ['2021-05-25T', '2021-05-26T', '2021-05-27T', '2021-06-01T', '2021-06-07T', '2021-06-08T']
1 ['2021-08-02T', '2021-08-03T', '2021-08-04T', '2021-08-05T', '2021-08-13T', '2021-08-17T', '2021-08-25T', '2021-08-31T', '2021-09-02T', '2021-09-06T']
2 ['2021-09-22T', '2021-09-23T', '2021-09-24T']
3 ['2021-10-04T', '2021-10-05T', '2021-10-08T', '2021-10-13T', '2021-10-19T']
4 ['2021-12-16T', '2021-12-17T', '2021-12-21T', '2021-12-23T', '2021-12-28T', '2021-12-30T', '2022-01-03T']
5 ['2022-01-28T', '2022-02-01T', '2022-02-02T']
6 ['2022-03-08T', '2022-03-09T', '2022-03-14T']
7 ['2022-04-01T', '2022-04-04T', '2022-04-05T']
8 ['2022-07-18T', '2022-07-19T', '2022-07-20T']
9 ['2022-07-28T', '2022-07-29T', '2022-08-01T', '2022-08-04T', '2022-08-05T']
10 ['2022-09-07T', '2022-09-08T', '2022-09-09T', '2022-09-12T']
11 ['2022-10-31T', '2022-11-01T', '2022-11-02T']
12 ['2022-11-07T', '2022-11-09T', '2022-11-11T']
13 ['2022-11-24T', '2022-11-25T', '2022-11-28T', '2022-12-01T']
14 ['2022-12-27T', '2022

In [11]:
Upeak = False
Lpeak = False
Upeak60 =False
count = 0
for ei, i in enumerate(df["rsi"]):
    if(Upeak==False) and (i>69):
        Upeak = True
        print("sdate=", df["date"][ei])
        count +=1
    if(Lpeak==False) and (i<31):
        Lpeak = True
    if Upeak and (i<60):
        Upeak = False
print(count)

sdate= 2021-01-04T13:45:00+0530
sdate= 2021-01-08T15:00:00+0530
sdate= 2021-01-12T13:00:00+0530
sdate= 2021-01-18T14:00:00+0530
sdate= 2021-01-19T09:15:00+0530
sdate= 2021-01-20T13:00:00+0530
sdate= 2021-02-02T14:30:00+0530
sdate= 2021-02-08T09:45:00+0530
sdate= 2021-02-09T09:30:00+0530
sdate= 2021-02-10T15:15:00+0530
sdate= 2021-02-25T09:15:00+0530
sdate= 2021-03-03T14:15:00+0530
sdate= 2021-03-05T10:00:00+0530
sdate= 2021-03-08T10:00:00+0530
sdate= 2021-03-19T14:15:00+0530
sdate= 2021-03-23T10:30:00+0530
sdate= 2021-03-30T10:45:00+0530
sdate= 2021-04-07T09:15:00+0530
sdate= 2021-04-15T15:00:00+0530
sdate= 2021-04-26T09:30:00+0530
sdate= 2021-04-27T09:15:00+0530
sdate= 2021-04-29T09:15:00+0530
sdate= 2021-05-07T09:15:00+0530
sdate= 2021-05-17T09:45:00+0530
sdate= 2021-05-19T11:45:00+0530
sdate= 2021-05-28T09:30:00+0530
sdate= 2021-06-01T09:30:00+0530
sdate= 2021-06-02T15:15:00+0530
sdate= 2021-06-07T13:00:00+0530
sdate= 2021-06-10T11:45:00+0530
sdate= 2021-06-14T13:45:00+0530
sdate= 2

In [49]:
stack = []
count = 0
first_set= {}
for ei, i in enumerate(df["rsi"]):
    if (ei>1984):
        close = df.iloc[ei]["close"]
        tmp = ((abs(df.iloc[ei]["close"]-df.iloc[ei+1:ei+501]["close"]))/df.iloc[ei]["close"])*100
        tmp_qualified =[]
        for et, t in enumerate(tmp):
            if (t> ((et+1)/10) and t >=3 and (len(tmp_qualified)==0 or t>tmp_qualified[0])):
                tmp_qualified=(t, df.iloc[ei+et+1]["date"]) 
        
        if (len(tmp_qualified)):
            if (tmp_qualified[-1] not in first_set.keys()):
                first_set[tmp_qualified[-1]]= (df.iloc[ei]["date"], tmp_qualified[0])
            else:
                if (tmp_qualified[0]>first_set[tmp_qualified[-1]][1]):
                    first_set[tmp_qualified[-1]]= (df.iloc[ei]["date"], tmp_qualified[0])
    

In [59]:
from dateutil import parser
clean_set ={}
count = 0
for ef, f in enumerate(first_set):
#     print((f, first_set[f]), clean_set, count)
    end_time = parser.parse(f)
    start_time = parser.parse(first_set[f][0])
    if count not in clean_set.keys():
            clean_set[count] = (first_set[f][0],f, first_set[f][1])
    else:
        if(((parser.parse(clean_set[count][1])-start_time)/(60*15))/((end_time-start_time)/(60*15)))<0.3:
            count+=1

            
            
            clean_set[count]=(first_set[f][0],f, first_set[f][1])
        else:
            if (first_set[f][1]>clean_set[count][2]):
                clean_set[count]=(first_set[f][0],f, first_set[f][1])
print(clean_set)

{0: ('2021-04-30T14:15:00+0530', '2021-05-04T10:45:00+0530', 3.632714500752639), 1: ('2021-05-04T14:45:00+0530', '2021-05-10T14:30:00+0530', 10.44760510353481), 2: ('2021-05-11T15:15:00+0530', '2021-05-14T12:45:00+0530', 5.3307766059443935), 3: ('2021-05-17T14:00:00+0530', '2021-05-19T09:45:00+0530', 3.472083249949963), 4: ('2021-05-25T15:00:00+0530', '2021-05-27T15:00:00+0530', 5.160417471975269), 5: ('2021-06-17T15:15:00+0530', '2021-06-21T09:45:00+0530', 4.076888690210095), 6: ('2021-06-21T12:00:00+0530', '2021-06-22T14:30:00+0530', 3.5185185185185186), 7: ('2021-06-22T14:30:00+0530', '2021-06-23T15:00:00+0530', 3.389982110912352), 8: ('2021-07-13T14:00:00+0530', '2021-07-19T09:15:00+0530', 11.506456241032994), 9: ('2021-07-19T09:15:00+0530', '2021-07-20T11:30:00+0530', 3.542631669240003), 10: ('2021-07-20T11:30:00+0530', '2021-07-23T14:30:00+0530', 6.9542018674966695), 11: ('2021-08-16T12:30:00+0530', '2021-08-18T10:00:00+0530', 4.170073589533932), 12: ('2021-09-02T10:30:00+0530', 

In [60]:
for f in first_set:
    print(f, first_set[f])

2021-05-04T10:45:00+0530 ('2021-04-30T14:15:00+0530', 3.632714500752639)
2021-05-04T10:15:00+0530 ('2021-04-30T14:30:00+0530', 3.428169297275563)
2021-05-06T13:45:00+0530 ('2021-05-04T10:00:00+0530', 6.568281022656416)
2021-05-07T09:15:00+0530 ('2021-05-04T10:45:00+0530', 7.237321670311362)
2021-05-10T14:30:00+0530 ('2021-05-04T14:45:00+0530', 10.44760510353481)
2021-05-10T09:15:00+0530 ('2021-05-05T14:30:00+0530', 5.410352893318422)
2021-05-14T10:15:00+0530 ('2021-05-11T14:00:00+0530', 3.5478867778208523)
2021-05-14T12:00:00+0530 ('2021-05-11T14:15:00+0530', 4.100824042656321)
2021-05-14T12:15:00+0530 ('2021-05-11T14:30:00+0530', 4.169899146625291)
2021-05-14T12:45:00+0530 ('2021-05-11T15:15:00+0530', 5.3307766059443935)
2021-05-19T09:45:00+0530 ('2021-05-17T14:00:00+0530', 3.472083249949963)
2021-05-27T09:45:00+0530 ('2021-05-25T14:00:00+0530', 4.396668603525092)
2021-05-27T15:00:00+0530 ('2021-05-25T15:00:00+0530', 5.160417471975269)
2021-06-21T09:45:00+0530 ('2021-06-17T15:15:00+05

In [62]:
for c in clean_set:
    print(clean_set[c])


('2021-04-30T14:15:00+0530', '2021-05-04T10:45:00+0530', 3.632714500752639)
('2021-05-04T14:45:00+0530', '2021-05-10T14:30:00+0530', 10.44760510353481)
('2021-05-11T15:15:00+0530', '2021-05-14T12:45:00+0530', 5.3307766059443935)
('2021-05-17T14:00:00+0530', '2021-05-19T09:45:00+0530', 3.472083249949963)
('2021-05-25T15:00:00+0530', '2021-05-27T15:00:00+0530', 5.160417471975269)
('2021-06-17T15:15:00+0530', '2021-06-21T09:45:00+0530', 4.076888690210095)
('2021-06-21T12:00:00+0530', '2021-06-22T14:30:00+0530', 3.5185185185185186)
('2021-06-22T14:30:00+0530', '2021-06-23T15:00:00+0530', 3.389982110912352)
('2021-07-13T14:00:00+0530', '2021-07-19T09:15:00+0530', 11.506456241032994)
('2021-07-19T09:15:00+0530', '2021-07-20T11:30:00+0530', 3.542631669240003)
('2021-07-20T11:30:00+0530', '2021-07-23T14:30:00+0530', 6.9542018674966695)
('2021-08-16T12:30:00+0530', '2021-08-18T10:00:00+0530', 4.170073589533932)
('2021-09-02T10:30:00+0530', '2021-09-06T14:45:00+0530', 6.738691123892433)
('2021-0

In [61]:
len(clean_set)

128

In [ ]:
def overlap(old_interval, new_interval):
    if (old_interval[0]<new_interval[1] and )

In [31]:
from dateutil import parser
clean_set ={}
count = 0
for ef, f in enumerate(first_set):
    
    end_time = parser.parse(f)
    start_time = parser.parse(first_set[f][0])
    if count not in clean_set.keys():
            clean_set[count] = (first_set[f][0],f, first_set[f][1])
    else:
        if(((parser.parse(clean_set[count][1])-start_time)/(60*15))/((end_time-start_time)/(60*15)))<0.1:
            count+=1
            clean_set[count]=(first_set[f][0],f, first_set[f][1])
        else:
            if (first_set[f][1]>clean_set[count][2]):
                clean_set[count]=(first_set[f][0],f, first_set[f][1])

    


In [32]:
clean_set

{0: ('2021-05-04T14:45:00+0530',
  '2021-05-10T14:30:00+0530',
  10.44760510353481),
 1: ('2021-05-11T15:15:00+0530',
  '2021-05-14T12:45:00+0530',
  5.3307766059443935),
 2: ('2021-05-17T14:00:00+0530',
  '2021-05-19T09:45:00+0530',
  3.472083249949963),
 3: ('2021-05-25T15:00:00+0530',
  '2021-05-27T15:00:00+0530',
  5.160417471975269),
 4: ('2021-06-17T15:15:00+0530',
  '2021-06-21T09:45:00+0530',
  4.076888690210095),
 5: ('2021-06-21T12:00:00+0530',
  '2021-06-22T14:30:00+0530',
  3.5185185185185186),
 6: ('2021-06-22T14:30:00+0530',
  '2021-06-23T15:00:00+0530',
  3.389982110912352),
 7: ('2021-07-13T14:00:00+0530',
  '2021-07-19T09:15:00+0530',
  11.506456241032994),
 8: ('2021-07-19T09:15:00+0530',
  '2021-07-20T11:30:00+0530',
  3.542631669240003),
 9: ('2021-07-20T11:30:00+0530',
  '2021-07-23T14:30:00+0530',
  6.9542018674966695),
 10: ('2021-08-16T12:30:00+0530',
  '2021-08-18T10:00:00+0530',
  4.170073589533932),
 11: ('2021-09-02T10:30:00+0530',
  '2021-09-06T14:45:00+053

In [60]:
count = {0:0, 10:0, 20:0, 30:0, 40:0, 50:0, 60:0, 70:0, 80:0, 90:0}
for c in clean_set:
    start_date = clean_set[c][0]
    end_date = clean_set[c][1]
    start_df = df.loc[(df['date'] == start_date)]
    end_df = df.loc[(df['date'] == end_date)]
    count[(start_df["rsi"].values[0]//10)*10]+=1
    if ((start_df["rsi"].values[0]//10)*10)==10:
        print(clean_set[c])
#     print(end_df["close"].values[0]-start_df["close"].values[0], end_df["rsi"].values[0])
print(count)

big_count = {0:0, 10:0, 20:0, 30:0, 40:0, 50:0, 60:0, 70:0, 80:0, 90:0}

('2022-02-14T15:00:00+0530', '2022-02-16T13:45:00+0530', 4.895621651579531)
('2022-06-17T12:00:00+0530', '2022-06-21T13:45:00+0530', 5.790258449304178)
('2022-07-15T12:30:00+0530', '2022-07-19T09:45:00+0530', 3.9376118639734026)
('2022-09-26T09:30:00+0530', '2022-09-27T09:15:00+0530', 3.882612647708087)
{0: 0, 10: 4, 20: 20, 30: 21, 40: 16, 50: 16, 60: 23, 70: 5, 80: 5, 90: 1}


In [65]:
count = {0:0, 10:0, 20:0, 30:0, 40:0, 50:0, 60:0, 70:0, 80:0, 90:0}

day_list = {0:[], 10:[], 20:[], 30:[], 40:[], 50:[], 60:[], 70:[], 80:[], 90:[]}
for c in range(len(df.date)):
    if (c>1984):
        prev_rsi = df.iloc[c-1]["rsi"]
        curr_rsi = df.iloc[c]["rsi"]
        
#         if (curr_rsi>=10 and curr_rsi<=20 and (prev_rsi<10 or prev_rsi>20) and df.iloc[c]["date"][:11] not in day_list):
#             day_list[10].append(df.iloc[c]["date"][:11])
#             print(df.iloc[c]["date"][:11])
#             count[10] +=1
#         if (curr_rsi>=20 and curr_rsi<=30 and (prev_rsi<20 or prev_rsi>30) and df.iloc[c]["date"][:11] not in day_list):
#             day_list[20].append(df.iloc[c]["date"][:11])
#             print(df.iloc[c]["date"][:11])
#             count[20] +=1
        for crt in range(0, 100, 10):
            if (curr_rsi>=crt and curr_rsi<=(crt+10) and (prev_rsi<crt or prev_rsi>(crt+10)) and df.iloc[c]["date"][:11] not in day_list):
                day_list[crt].append(df.iloc[c]["date"][:11])
#                 if crt ==10:
#                     print(df.iloc[c]["date"][:11])
                count[crt] +=1
print(count)

{0: 2, 10: 74, 20: 374, 30: 966, 40: 1546, 50: 1532, 60: 945, 70: 367, 80: 73, 90: 5}


In [63]:
#downrsi above up rsi
count_rsi_down_start =0
count_rsi_down_end = 0

last_rsi_down_start = [0]
last_rsi_down_end = [0]

last_rsi_up_start = [0]
last_rsi_up_end = [0]

upper_boundary=[(0,1)]
lower_boundary = [(0,1)]


stack_70 = []

for c in range(len(df.date)):
    if (c>1804):
        if (c==1805):
            print(df.iloc[c])
        prev_rsi = df.iloc[c-1]["rsi"]
        curr_rsi = df.iloc[c]["rsi"]
        if prev_rsi>=30 and curr_rsi<30:
#             if (c-last_rsi_down_start[-1])<15 :
#                 count_rsi_down_start +=1
#                 last_rsi_down_start.append(c)
#                 print(df.iloc[c])
#             else:
                last_rsi_down_start=[]
                last_rsi_down_start.append(c)
                
        if prev_rsi<=30 and curr_rsi>30 :
#             if (c-last_rsi_down_end[-1])>15:
#                 count_rsi_down_end +=1
#                 last_rsi_down_end.append(c)
#             else:
                last_rsi_down_end=[]
                last_rsi_down_end.append(c)
                lower_boundary.append((last_rsi_down_start[-1],last_rsi_down_end[-1]))
#                 print(df.iloc[lower_boundary[-1][0]], df.iloc[lower_boundary[-1][1]])

#                 if (abs(min(df.iloc[upper_boundary[-1][0]:upper_boundary[-1][1]]["low"])-min(df.iloc[lower_boundary[-1][0]:lower_boundary[-1][1]]["close"]))/min(min(df.iloc[upper_boundary[-1][0]:upper_boundary[-1][1]]["low"]),min(df.iloc[lower_boundary[-1][0]:lower_boundary[-1][1]]["close"])))<0.01:
#                     print(df.iloc[lower_boundary[-1][0]]["date"])
                
     
        if prev_rsi>=70 and curr_rsi<70:
#             if (c-last_rsi_up_end[-1])<15 :
#                 count_rsi_up_end +=1
#                 last_rsi_up_end.append(c)
#                 print(df.iloc[c])
#             else:
                last_rsi_up_end=[]
                last_rsi_up_end.append(c)
#                 print(df.iloc[c])
                upper_boundary.append((last_rsi_up_start[-1],last_rsi_up_end[-1]))
#                 print(df.iloc[upper_boundary[-1][0]], df.iloc[upper_boundary[-1][1]])
#                 break=
#                 print("lowerBOundary=",lower_boundary, upper_boundary)
#                 if (abs(max(df.iloc[lower_boundary[-1][0]:lower_boundary[-1][1]]["high"])-max(df.iloc[upper_boundary[-1][0]:upper_boundary[-1][1]]["close"]))/min(max(df.iloc[lower_boundary[-1][0]:lower_boundary[-1][1]]["high"]),max(df.iloc[upper_boundary[-1][0]:upper_boundary[-1][1]]["close"])))<0.01:
#                     print("up=",df.iloc[lower_boundary[-1][0]]["date"])
#                     print(df.iloc[upper_boundary[-1][0]]["date"])
                

     
        if prev_rsi<=70 and curr_rsi>70 :
#             if (c-last_rsi_up_start[-1])<15:
#                 count_rsi_up_start +=1
#                 last_rsi_up_start.append(c)
#             else:
                last_rsi_up_start=[]
                last_rsi_up_start.append(c)
                
print(count_rsi_down_end)

date      2021-04-20T14:15:00+0530
open                         472.8
high                         474.8
low                          469.0
close                       470.65
volume                     1158634
un                               0
rsi                      49.770812
Name: 1805, dtype: object
up= 2021-05-14T09:15:00+0530
2021-05-18T09:30:00+0530
up= 2021-07-07T09:45:00+0530
2021-07-14T10:15:00+0530
up= 2021-09-16T12:45:00+0530
2021-09-22T10:30:00+0530
up= 2021-09-29T09:15:00+0530
2021-10-04T15:15:00+0530
up= 2022-03-25T12:30:00+0530
2022-03-30T09:15:00+0530
up= 2022-05-11T10:15:00+0530
2022-05-17T10:30:00+0530
up= 2022-06-28T09:15:00+0530
2022-07-01T15:00:00+0530
up= 2022-07-15T09:15:00+0530
2022-07-18T09:45:00+0530
up= 2022-07-15T09:15:00+0530
2022-07-18T10:30:00+0530
up= 2022-07-15T09:15:00+0530
2022-07-18T15:15:00+0530
up= 2022-07-15T09:15:00+0530
2022-07-19T09:45:00+0530
up= 2022-07-15T09:15:00+0530
2022-07-19T10:45:00+0530
up= 2022-08-10T09:15:00+0530
2022-08-11T09:15:

In [37]:
if ((abs((min(df.iloc[upper_boundary[-1][0]:upper_boundary[-1][1]]["low"]))-
min(df.iloc[lower_boundary[-1][0]:lower_boundary[-1][1]]["close"]))/min(((min(df.iloc[upper_boundary[-1][0]:upper_boundary[-1][1]]["low"]))-
min(df.iloc[lower_boundary[-1][0]:lower_boundary[-1][1]]["close"]))))*100)<1:
    print(True)

TypeError: 'float' object is not iterable

In [ ]:
((abs((
min(df.iloc[upper_boundary[-1][0]:upper_boundary[-1][1]]["low"]))-
min(df.iloc[lower_boundary[-1][0]:lower_boundary[-1][1]]["close"]))/
min((
(min(df.iloc[upper_boundary[-1][0]:upper_boundary[-1][1]]["low"]))-
min(df.iloc[lower_boundary[-1][0]:lower_boundary[-1][1]]["close"]))))*100)

In [48]:
if (abs(min(df.iloc[upper_boundary[-1][0]:upper_boundary[-1][1]]["low"])-min(df.iloc[lower_boundary[-1][0]:lower_boundary[-1][1]]["close"]))/min(min(df.iloc[upper_boundary[-1][0]:upper_boundary[-1][1]]["low"]),min(df.iloc[lower_boundary[-1][0]:lower_boundary[-1][1]]["close"])))<0.01:
    print(True)

True


In [44]:
min(df.iloc[upper_boundary[-1][0]:upper_boundary[-1][1]]["low"])

477.2